First of all we need to download the data. We'll use the GTSRB Training Images Set as our whole Dataset and divide it later to a training and test set. In order to get this Notebook working, you need to do the following steps:
- Change the referencing folderpath to yours
- install opencv via the Terminal: pip3 install opencv-python

The Following tasks numbered in the cells (1., 2., 3., ...) are shown in the Table of Contents:

1. Importing Librarys
2. Downloading the Data
3. Renaming the Data
4. Extracting the HOG Features
5. Split the Data and Train the model and evaluating - HOG -Random Forest
6. Split the Data and Train the model and evaluating - HOG -Support Vector Machine
7. Extracting the HUE Features
8. Split the Data and Train the model and evaluating - HUE -Random Forest
9. Split the Data and Train the model and evaluating - HUE -Support Vector Machine


In [1]:
# 1.Importing Librarys
import requests
import zipfile
import os
import numpy as np
from skimage.io import imshow, imread
from skimage.feature import hog
from skimage import exposure
import skimage.transform as transform
import skimage.io
import skimage.exposure
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import make_classification
import cv2 # pls enter this command in your Terminal: pip3 install opencv-python

In [1]:
# 2.Downloading the Data
# URL of the dataset
url = "https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the downloaded file
    with open("dataset.zip", "wb") as file:
        file.write(response.content)
    
    # Extract the contents of the ZIP file
    with zipfile.ZipFile("dataset.zip", "r") as zip_ref:
        zip_ref.extractall("dataset_folder_train_images")
    
    print("Dataset downloaded and extracted successfully.")
else:
    print("Failed to download the dataset.")

Dataset downloaded and extracted successfully.


In [4]:
# 3.Renaming the Data
# Renaming the datainfrastructure
def rename_folders(folder_path):
    for folder_name in os.listdir(folder_path):
        folder_old_name = os.path.join(folder_path, folder_name)
        folder_new_name = os.path.join(folder_path, str(int(folder_name)))
        
        # Rename the folder
        os.rename(folder_old_name, folder_new_name)
        
# Be sure, to insert the right folderpath here!  
rename_folders('/home/jovyan/ML Projekkkkt/App.ML/dataset_folder_train_images/GTSRB/Final_Training/Images/')

In [2]:
# 4.Extracting the HOG Features
#read the dataset and put it in an array for HOG_features:
def read_data_HOG(folder_path):
    X = []
    y = []

    for class_label in range(43):
        class_folder = os.path.join(folder_path, str(class_label))
        
        for file_name in os.listdir(class_folder):
            if file_name.endswith(".ppm"):
                file_path = os.path.join(class_folder, file_name)
                
                # Code snippet for hogz
                img = skimage.io.imread(file_path, plugin='matplotlib')
                img_gray = skimage.color.rgb2gray(img)
                reshaped_img = transform.resize(img_gray, (40, 40))
                normalized_image = skimage.exposure.rescale_intensity(reshaped_img, in_range='image', out_range=(0, 1))
                hog_features, hog_image = hog(normalized_image, orientations=8, pixels_per_cell=(6, 6), cells_per_block=(2, 2), visualize=True)
                hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 5))
                hog_features_array = hog_features.reshape(-1)
                X.append(hog_features_array)
                y.append(class_label)

    return np.array(X), np.array(y)

folder_path_train = "/home/jovyan/ML Projekkkkt/App.ML/dataset_folder_train_images/GTSRB/Final_Training/Images"

X_hog,y_hog = read_data_HOG(folder_path_train)

In [3]:
# 5.Split the Data and Train the model and evaluating - HOG - Random Forest
#random Forest for HOG_Features
X_train_forest_hog, X_test_forest_hog, y_train_forest_hog, y_test_forest_hog = train_test_split(X_hog, y_hog, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_forest_hog = scaler.fit_transform(X_train_forest_hog)
X_test_scaled_forest_hog = scaler.transform(X_test_forest_hog)

# Create and train the Random Forest model
random_forest_hog = RandomForestClassifier()
random_forest_hog.fit(X_train_scaled_forest_hog, y_train_forest_hog)

# Make predictions on the test set
y_pred_forest_hog = random_forest_hog.predict(X_test_scaled_forest_hog)

# Evaluate the model
accuracy_forest_hog = accuracy_score(y_test_forest_hog, y_pred_forest_hog)
print("Accuracy for Random Forest:", accuracy_forest_hog)
print()

# Calculating precision, recall, and F1-score for each class
precision_forest_hog = precision_score(y_test_forest_hog, y_pred_forest_hog, average=None)
recall_forest_hog = recall_score(y_test_forest_hog, y_pred_forest_hog, average=None)
f1_forest_hog = f1_score(y_test_forest_hog, y_pred_forest_hog, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score_rf1 in zip(np.unique(y_hog), precision_forest_hog, recall_forest_hog, f1_forest_hog):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score_rf1}")
    print()

Accuracy for Random Forest: 0.9435093088497832

Class: 0
Precision: 1.0
Recall: 0.5263157894736842
F1-Score: 0.6896551724137931

Class: 1
Precision: 0.9104166666666667
Recall: 0.8810483870967742
F1-Score: 0.8954918032786885

Class: 2
Precision: 0.8178137651821862
Recall: 0.8977777777777778
F1-Score: 0.8559322033898304

Class: 3
Precision: 0.9471698113207547
Recall: 0.8964285714285715
F1-Score: 0.9211009174311928

Class: 4
Precision: 0.9049773755656109
Recall: 0.9569377990430622
F1-Score: 0.9302325581395349

Class: 5
Precision: 0.7950138504155124
Recall: 0.7884615384615384
F1-Score: 0.7917241379310345

Class: 6
Precision: 0.9672131147540983
Recall: 1.0
F1-Score: 0.9833333333333333

Class: 7
Precision: 0.887719298245614
Recall: 0.9100719424460432
F1-Score: 0.8987566607460035

Class: 8
Precision: 0.901060070671378
Recall: 0.8471760797342193
F1-Score: 0.8732876712328768

Class: 9
Precision: 0.9671532846715328
Recall: 0.9888059701492538
F1-Score: 0.977859778597786

Class: 10
Precision: 0.96

In [4]:
# 6.Split the Data and Train the model and evaluating - HOG - Support Vector Machine
#SVM for HOG_Features
X_train_svm_hog, X_test_svm_hog, y_train_svm_hog, y_test_svm_hog = train_test_split(X_hog, y_hog, test_size=0.2, random_state=42)

clf_hog= make_pipeline(StandardScaler(),SVC(kernel="rbf", gamma= "auto"))
clf_hog.fit(X_train_svm_hog,y_train_svm_hog)
y_pred_svm_hog = clf_hog.predict(X_test_svm_hog)
accuracy_svm_hog = accuracy_score(y_test_svm_hog, y_pred_svm_hog)
print("Accuracy for a SVM:", accuracy_svm_hog)
print()

# Calculating precision, recall, and F1-score for each class
precision_svm_hog = precision_score(y_test_svm_hog, y_pred_svm_hog, average=None)
recall_svm_hog = recall_score(y_test_svm_hog, y_pred_svm_hog, average=None)
f1_svm_hog = f1_score(y_test_svm_hog, y_pred_svm_hog, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score_svm1  in zip(np.unique(y_hog), precision_svm_hog, recall_svm_hog, f1_svm_hog):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score_svm1}")
    print()

Accuracy for a SVM: 0.9802346340219332

Class: 0
Precision: 1.0
Recall: 0.9210526315789473
F1-Score: 0.958904109589041

Class: 1
Precision: 0.945010183299389
Recall: 0.9354838709677419
F1-Score: 0.9402228976697062

Class: 2
Precision: 0.9166666666666666
Recall: 0.9533333333333334
F1-Score: 0.934640522875817

Class: 3
Precision: 0.9818181818181818
Recall: 0.9642857142857143
F1-Score: 0.972972972972973

Class: 4
Precision: 0.9809976247030879
Recall: 0.9880382775119617
F1-Score: 0.9845053635280095

Class: 5
Precision: 0.9078212290502793
Recall: 0.8928571428571429
F1-Score: 0.9002770083102493

Class: 6
Precision: 0.9833333333333333
Recall: 1.0
F1-Score: 0.9915966386554621

Class: 7
Precision: 0.9853479853479854
Recall: 0.9676258992805755
F1-Score: 0.9764065335753176

Class: 8
Precision: 0.9634551495016611
Recall: 0.9634551495016611
F1-Score: 0.9634551495016611

Class: 9
Precision: 1.0
Recall: 0.996268656716418
F1-Score: 0.9981308411214954

Class: 10
Precision: 0.9973045822102425
Recall: 1.

In [5]:
# 7.Extracting the HUE Features
def read_data_HUE(folder_path):
    X = []
    y = []
    
    desired_width = 29
    desired_height = 30

    for class_label in range(43):
        class_folder = os.path.join(folder_path, str(class_label))
        
        
        
        for file_name in os.listdir(class_folder):
            if file_name.endswith(".ppm"):
                file_path = os.path.join(class_folder, file_name)
                
                
            
                image = cv2.imread(file_path)
                scaled_image = cv2.resize(image, (desired_width, desired_height)) 
                image_hsv = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2HSV) 
                
                border_size = 5
                image_hsv_cropped = image_hsv[border_size:-border_size, border_size:-border_size] 
                hue_values = image_hsv_cropped[:, :, 0].flatten()
                
                histogram, _ = np.histogram(hue_values, bins=256, range=[0, 256])
                normalized_histogram = histogram.astype(np.float64) / np.sum(histogram)
                dimensionality = len(normalized_histogram)
               
                
                X.append(hue_values)
                y.append(class_label)

    return np.array(X), np.array(y)

X_hue,y_hue = read_data_HUE(folder_path_train)

In [6]:
# 8.Split the Data and Train the model and evaluating - HUE - Random Forest
#random Forest for HUE features
X_train_forest_hue, X_test_forest_hue, y_train_forest_hue, y_test_forest_hue = train_test_split(X_hue, y_hue, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_forest_hue = scaler.fit_transform(X_train_forest_hue)
X_test_scaled_forest_hue = scaler.transform(X_test_forest_hue)

# Create and train the Random Forest model
random_forest_hue = RandomForestClassifier()
random_forest_hue.fit(X_train_scaled_forest_hue, y_train_forest_hue)

# Make predictions on the test set
y_pred_forest_hue = random_forest_hue.predict(X_test_scaled_forest_hue)

# Evaluate the model
accuracy_forest_hue = accuracy_score(y_test_forest_hue, y_pred_forest_hue)
print("Accuracy for Random Forest:", accuracy_forest_hue)
print()

# Calculating precision, recall, and F1-score for each class
precision_forest_hue = precision_score(y_test_forest_hue, y_pred_forest_hue, average=None)
recall_forest_hue = recall_score(y_test_forest_hue, y_pred_forest_hue, average=None)
f1_forest_hue = f1_score(y_test_forest_hue, y_pred_forest_hue, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score_rf2 in zip(np.unique(y_hue), precision_forest_hue, recall_forest_hue, f1_forest_hue):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score_rf2}")
    print()

Accuracy for Random Forest: 0.7510839071665392

Class: 0
Precision: 1.0
Recall: 0.2894736842105263
F1-Score: 0.4489795918367347

Class: 1
Precision: 0.6407942238267148
Recall: 0.7157258064516129
F1-Score: 0.6761904761904762

Class: 2
Precision: 0.42191435768261965
Recall: 0.7444444444444445
F1-Score: 0.5385852090032154

Class: 3
Precision: 0.6906077348066298
Recall: 0.44642857142857145
F1-Score: 0.5422993492407809

Class: 4
Precision: 0.6512820512820513
Recall: 0.6076555023923444
F1-Score: 0.6287128712871287

Class: 5
Precision: 0.5446927374301676
Recall: 0.5357142857142857
F1-Score: 0.5401662049861496

Class: 6
Precision: 0.7321428571428571
Recall: 0.6949152542372882
F1-Score: 0.7130434782608696

Class: 7
Precision: 0.6304347826086957
Recall: 0.5215827338129496
F1-Score: 0.5708661417322834

Class: 8
Precision: 0.678714859437751
Recall: 0.5614617940199336
F1-Score: 0.6145454545454546

Class: 9
Precision: 0.9338842975206612
Recall: 0.8432835820895522
F1-Score: 0.8862745098039215

Class:

In [7]:
# 9.Split the Data and Train the model and evaluating - HUE - Support Vector Machine
#SVM for HUE features
X_train_svm_hue, X_test_svm_hue, y_train_svm_hue, y_test_svm_hue = train_test_split(X_hue, y_hue, test_size=0.2, random_state=42)

clf_hue= make_pipeline(StandardScaler(),SVC(kernel="rbf", gamma= "auto"))
clf_hue.fit(X_train_svm_hue,y_train_svm_hue)
y_pred_svm_hue = clf_hue.predict(X_test_svm_hue)
accuracy_svm_hue = accuracy_score(y_test_svm_hue, y_pred_svm_hue)
print("Accuracy for a SVM:", accuracy_svm_hue)
print()

# Calculating precision, recall, and F1-score for each class
precision_svm_hue = precision_score(y_test_svm_hue, y_pred_svm_hue, average=None)
recall_svm_hue = recall_score(y_test_svm_hue, y_pred_svm_hue, average=None)
f1_svm_hue = f1_score(y_test_svm_hue, y_pred_svm_hue, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score_svm2 in zip(np.unique(y_hue), precision_svm_hue, recall_svm_hue, f1_svm_hue):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score_svm2}")
    print()

Accuracy for a SVM: 0.5795715378729915

Class: 0
Precision: 1.0
Recall: 0.18421052631578946
F1-Score: 0.3111111111111111

Class: 1
Precision: 0.5649819494584838
Recall: 0.6310483870967742
F1-Score: 0.5961904761904763

Class: 2
Precision: 0.41420118343195267
Recall: 0.6222222222222222
F1-Score: 0.4973357015985791

Class: 3
Precision: 0.47305389221556887
Recall: 0.28214285714285714
F1-Score: 0.35346756152125286

Class: 4
Precision: 0.432967032967033
Recall: 0.47129186602870815
F1-Score: 0.45131729667812137

Class: 5
Precision: 0.32387706855791965
Recall: 0.37637362637362637
F1-Score: 0.34815756035578144

Class: 6
Precision: 0.5714285714285714
Recall: 0.3389830508474576
F1-Score: 0.42553191489361697

Class: 7
Precision: 0.44019138755980863
Recall: 0.33093525179856115
F1-Score: 0.37782340862423

Class: 8
Precision: 0.3611111111111111
Recall: 0.34551495016611294
F1-Score: 0.3531409168081494

Class: 9
Precision: 0.7154150197628458
Recall: 0.6753731343283582
F1-Score: 0.6948176583493283

Clas